In [3]:
import keras
from keras.models import load_model
from keras.models import Model
print("keras version:", keras.__version__)
import tensorflow as tf
print("tensorflow version:", tf.__version__)
import wget
import os
import sys
from keras.datasets import cifar10
import numpy as np
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train, axis=(0, 1, 2, 3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)
x_valid = x_train[:10000]
y_valid = y_train[:10000]

keras version: 2.3.1
tensorflow version: 1.15.0


In [6]:
model_files = [
    "model_cifar10_balanced_seed-0_bestbefore-100_currentepoch-100_valacc-91_vgg.h5",
    "model_cifar10_balanced_seed-10_bestbefore-100_currentepoch-100_valacc-91_vgg.h5",
    "model_cifar10_balanced_seed-20_bestbefore-100_currentepoch-100_valacc-91_vgg.h5",
    "model_cifar10_balanced_seed-30_bestbefore-100_currentepoch-100_valacc-91_vgg.h5",
    "model_cifar10_balanced_seed-40_bestbefore-100_currentepoch-100_valacc-91_vgg.h5",
    "model_cifar10_balanced_seed-50_bestbefore-100_currentepoch-100_valacc-91_vgg.h5",
    "model_cifar10_balanced_seed-60_bestbefore-100_currentepoch-100_valacc-91_vgg.h5",
    "model_cifar10_balanced_seed-70_bestbefore-100_currentepoch-100_valacc-91_vgg.h5",
    "model_cifar10_balanced_seed-80_bestbefore-100_currentepoch-100_valacc-90_vgg.h5",
    "model_cifar10_balanced_seed-90_bestbefore-100_currentepoch-100_valacc-91_vgg.h5"
]
model_file = model_files[0] # test with 1 model

print("On model", model_file)
#   if (os.path.isfile(model_file)==False):
#     print("Downloading", model_file)
#     wget.download("https://zenodo.org/record/2648107/files/"
#                   +model_file+"?download=1", out=model_file)
model = load_model("models/" + model_file)

pre_softmax_model = Model(input=model.input,
                        output=model.layers[-2].output)
print("Making predictions on validation set")
valid_preacts = pre_softmax_model.predict(x_valid)
print("Making predictions on test set")
test_preacts = pre_softmax_model.predict(x_test)

print("Test accuracy",np.mean(np.argmax(test_preacts,axis=1)
                            == np.squeeze(y_test)))
print("Valid accuracy",np.mean(np.argmax(valid_preacts,axis=1)
                             == np.squeeze(y_valid)))

On model model_cifar10_balanced_seed-0_bestbefore-100_currentepoch-100_valacc-91_vgg.h5
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Making predictions on validation set



/Users/jsu/opt/anaconda3/lib/python3.7/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
/Users/jsu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("co..., outputs=Tensor("de...)`


Making predictions on test set
Test accuracy 0.8967
Valid accuracy 0.9054


In [13]:
from deepexplain.tensorflow import DeepExplain
from keras import backend as K

with DeepExplain(session=K.get_session()) as de:  # <-- init DeepExplain context
    # Need to reconstruct the graph in DeepExplain context, using the same weights.
    # With Keras this is very easy:
    # 1. Get the input tensor to the original model
    input_tensor = model.input # model.layers[0].input
    print(input_tensor)
    
    # 2. We now target the output of the last dense layer (pre-softmax)
    # To do so, create a new model sharing the same layers untill the last dense (index -2)
    fModel = Model(inputs=input_tensor, outputs = model.layers[-2].output)
    target_tensor = fModel(input_tensor)
    
    print(target_tensor)
    #y_train = keras.utils.to_categorical(y_train, num_classes)
    num_classes = 10
    y_test = keras.utils.to_categorical(y_test, num_classes)
    xs = x_test[0:10]
    ys = y_test[0:10]
    
    print(np.array(xs).shape)
    print(np.array(ys).shape)
    
    #attributions_gradin = de.explain('grad*input', target_tensor, input_tensor, xs, ys=ys)
    #attributions_sal   = de.explain('saliency', target_tensor, input_tensor, xs, ys=ys)
    #attributions_ig    = de.explain('intgrad', target_tensor, input_tensor, xs, ys=ys)
    #attributions_dl    = de.explain('deeplift', target_tensor, input_tensor, xs, ys=ys)
    attributions_idl   = de.explain('idl_true', target_tensor, input_tensor, xs, ys=ys)

Tensor("conv2d_1_input:0", shape=(?, 32, 32, 3), dtype=float32)
Tensor("model_6/dense_2/BiasAdd:0", shape=(?, 10), dtype=float32)
(10, 32, 32, 3)
(10, 10)
[array([[[[ 0.35747126,  1.416216  ,  0.77575797, ..., -1.2182026 ,
          -0.63238007,  0.26986519],
         [ 0.35564098,  1.4162066 ,  0.7759768 , ..., -1.214101  ,
          -0.6323799 ,  0.27388868],
         [ 0.35502362,  1.4162794 ,  0.776168  , ..., -1.2142363 ,
          -0.63238025,  0.27428964],
         ...,
         [ 0.35558915,  1.4158552 ,  0.77496976, ..., -1.2120932 ,
          -0.632379  ,  0.2722295 ],
         [ 0.35604966,  1.4157602 ,  0.77468705, ..., -1.2115494 ,
          -0.63237864,  0.27106822],
         [ 0.35925862,  1.4160272 ,  0.77569264, ..., -1.2105713 ,
          -0.63237894,  0.2676295 ]],

        [[ 0.35763448,  1.4162045 ,  0.7754976 , ..., -1.2164006 ,
          -0.6323799 ,  0.27194226],
         [ 0.35757953,  1.4161849 ,  0.77586263, ..., -1.210741  ,
          -0.6323795 ,  0.2770462

Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


ValueError: Cannot feed value of shape (10, 32, 32, 64) for Tensor 'Variable_42/initial_value:0', which has shape '(1, 32, 32, 64)'